<a href="https://colab.research.google.com/github/Shreya2506/ShoppingCart/blob/master/RSSI_SVM_MULTIOUT_WIFI_PRO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU Settings

*   Test Whether GPU is running
*   Which GPU are we using? 


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

*   Check the runtime

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

# The SVR code for Wifi localization

Importing Libraries
---

In [ ]:
from pathlib import Path
from tensorflow import keras
from PIL import Image as im

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from sklearn import svm
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import MinMaxScaler


import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as npC
import os
import seaborn as sns
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Data Import from CSV File
---

In [ ]:
Data = pd.read_csv('/content/drive/MyDrive/workfiles_Kaggle/5da138364db8ce0c98bc00f1/F1/ExcelFiles/WIFI_RSSMap/Mean/RSSI_final_v3_ForMulti.csv')

Data Preparation
---


In [ ]:
"""
# Printing the Data shape
"""
print(Data.shape)
Data.info()

"""
# Printing the names of columns
"""
for col_name in Data.columns: 
    print(col_name)
print(f'############')

In [ ]:
"""
# Data = features
"""

Data = Data.drop('Time', axis=1)
Data = Data.drop('TimeStamp', axis=1)
Data = Data.drop('Location', axis=1)
Data = Data.drop('x', axis=1)
Data = Data.drop('y', axis=1)


Data_x = np.array(Data["x.1"])
unique_elements, counts_elements = np.unique(Data_x, return_counts=True)
array = np.asarray((unique_elements, counts_elements))
array = array.transpose()
# print(array)
index = 0
for col in array:
    if array[index][1] > 1:
        print(array[index][0])

        index = index + 1
    else:
        index = index + 1
print(f'The values in location are unique')
print(Data.shape)
print(Data.count)


In [ ]:
"""
# X = Features
# y = Location to be predicted (contains 2 labels 'x' and 'y')
"""
X = Data
X = X.drop('x.1', axis=1)
X = X.drop('y.1', axis=1)
print(X.shape)
y = Data[Data.columns[-2:]]
print(y.shape)


In [ ]:
"""
Converting to nparray
"""
X = np.array(X)
y = np.array(y)

In [ ]:
"""
# Feature scaling
"""
sc_X = StandardScaler()
sc_Y = StandardScaler()
X = sc_X.fit_transform(X)
y = sc_Y.fit_transform(y)

In [ ]:
"""
# Normalization
"""
# create scaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

In [ ]:
from sklearn.preprocessing import RobustScaler

transx = RobustScaler()
transy = RobustScaler()
X = transx.fit_transform(X)
y = transy.fit_transform(y)

In [ ]:
"""
# Splitting into Train and Test data
"""
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

Model
---

In [ ]:
"""
# Creating a pipeline to implement GridSearchCV for multiout regressor.
"""
pipe_svr = Pipeline([
        ('reg', MultiOutputRegressor(SVR())
        )])

grid_param_svr = {
    'reg__estimator__C': [0.1,1,10,100,1000,10000],
    'reg__estimator__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'reg__estimator__epsilon': [0.01,0.02,0.06,0.07,0.08,0.09,0.1,0.5,0.03]}

gs_svr = (GridSearchCV(estimator=pipe_svr, 
                      param_grid=grid_param_svr, 
                      scoring = 'neg_mean_squared_error',
                      cv = 5,
                      verbose = 1,
                      n_jobs = -1))

gs_svr = gs_svr.fit(X_train,y_train)
gs_svr.best_estimator_ 

Tesing
---

In [ ]:
"""
# Testing the models using CV=5
"""

model = svm.SVR(kernel='rbf', C=100.0, gamma=0.0001, epsilon= 0.01)
wrapper = MultiOutputRegressor(model)
scores = cross_validate(wrapper, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(scores['test_score'].mean())

model2 = svm.SVR(kernel='rbf', C=100.0, gamma=0.0001, epsilon= 0.02)
wrapper2 = MultiOutputRegressor(model2)
scores2 = cross_validate(wrapper2, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(scores2['test_score'].mean())

model3 = svm.SVR(kernel='rbf', C=100.0, gamma=0.0001, epsilon= 0.03)
wrapper3 = MultiOutputRegressor(model3)
scores3 = cross_validate(wrapper3, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(scores2['test_score'].mean())

model4 = svm.SVR(kernel='rbf', C=100.0, gamma=0.0001, epsilon= 0.04)
wrapper4 = MultiOutputRegressor(model4)
scores4 = cross_validate(wrapper4, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(scores4['test_score'].mean())

model5 = svm.SVR(kernel='rbf', C=100.0, gamma=0.0001, epsilon=0.05)
wrapper5 = MultiOutputRegressor(model5)
scores5 = cross_validate(wrapper5, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(scores5['test_score'].mean())

In [ ]:
"""
# Predictions of different models (For constructing graphs)
"""
model = svm.SVR(kernel='rbf', C=1.0, gamma=0.0001, epsilon= 0.03)
wrapper = MultiOutputRegressor(model)
wrapper.fit(X_train, y_train)
predictions_e_1 = wrapper.predict(X_test)

model2 = svm.SVR(kernel='rbf', C=10.0, gamma=0.0001, epsilon= 0.03)
wrapper2 = MultiOutputRegressor(model2)
wrapper2.fit(X_train, y_train)
predictions_e_2 = wrapper2.predict(X_test)

model3 = svm.SVR(kernel='rbf', C=100.0, gamma=0.0001, epsilon= 0.03)
wrapper3 = MultiOutputRegressor(model3)
wrapper3.fit(X_train, y_train)
predictions_e_3 = wrapper3.predict(X_test)

model4 = svm.SVR(kernel='rbf', C=1000.0, gamma=0.0001, epsilon= 0.03)
wrapper4 = MultiOutputRegressor(model4)
wrapper4.fit(X_train, y_train)
predictions_e_4 = wrapper4.predict(X_test)

model5 = svm.SVR(kernel='rbf', C=10000.0, gamma=0.0001, epsilon=0.03)
wrapper5 = MultiOutputRegressor(model5)
wrapper5.fit(X_train, y_train)
predictions_e_5 = wrapper5.predict(X_test)


In [ ]:
"""
# Predictions of different models (For constructing graphs)
"""

model6 = svm.SVR(kernel='rbf', C=10.0, gamma=0.0001, epsilon= 0.06)
wrapper6 = MultiOutputRegressor(model6)
wrapper6.fit(X_train, y_train)
predictions_e_6 = wrapper6.predict(X_test)

model7 = svm.SVR(kernel='rbf', C=10.0, gamma=0.0001, epsilon= 0.07)
wrapper7 = MultiOutputRegressor(model7)
wrapper7.fit(X_train, y_train)
predictions_e_7 = wrapper7.predict(X_test)

model8 = svm.SVR(kernel='rbf', C=10.0, gamma=0.0001, epsilon= 0.08)
wrapper8 = MultiOutputRegressor(model8)
wrapper8.fit(X_train, y_train)
predictions_e_8 = wrapper8.predict(X_test)

model9 = svm.SVR(kernel='rbf', C=10.0, gamma=0.0001, epsilon= 0.09)
wrapper9 = MultiOutputRegressor(model9)
wrapper9.fit(X_train, y_train)
predictions_e_9 = wrapper9.predict(X_test)

model10 = svm.SVR(kernel='rbf', C=10.0, gamma=0.0001, epsilon=0.1)
wrapper10 = MultiOutputRegressor(model10)
wrapper10.fit(X_train, y_train)
predictions_e_10 = wrapper10.predict(X_test)

Save Models
---

In [ ]:
# Save the model
import os.path

if os.path.isfile('/content/drive/MyDrive/workfiles_Kaggle/5da138364db8ce0c98bc00f1/F1/ExcelFiles/WIFI_RSSMap/PickledObjects/model_100.pkl') is False:
  best_model.save('/content/drive/MyDrive/workfiles_Kaggle/5da138364db8ce0c98bc00f1/F1/ExcelFiles/WIFI_RSSMap/PickledObjects/model_100.pkl')

Result
---

In [ ]:
"""
# Inversing the Transformed data
"""
predictions_e_1 = sc_Y.inverse_transform(predictions_e_1)
predictions_e_2 = sc_Y.inverse_transform(predictions_e_2)
predictions_e_3 = sc_Y.inverse_transform(predictions_e_3)
predictions_e_4 = sc_Y.inverse_transform(predictions_e_4)
predictions_e_5 = sc_Y.inverse_transform(predictions_e_5)

In [ ]:
predictions_e_6 = sc_Y.inverse_transform(predictions_e_6)
predictions_e_7 = sc_Y.inverse_transform(predictions_e_7)
predictions_e_8 = sc_Y.inverse_transform(predictions_e_8)
predictions_e_9 = sc_Y.inverse_transform(predictions_e_9)
predictions_e_10 = sc_Y.inverse_transform(predictions_e_10)

In [ ]:
predictions_e_2 = scaler.inverse_transform(predictions_e_2)
y_test = scaler.inverse_transform(y)

In [ ]:
predictions_e_2 = transy.inverse_transform(predictions_e_2)
y_test = transy.inverse_transform(y_test)

In [ ]:
"""
# Inversing the Transformed data
"""
y_test = sc_Y.inverse_transform(y_test)

In [ ]:
"""
# Printing the result side by side
"""
# result = np.concatenate([y_test,predictions_e_2], axis=1)
# print(result)

"""
# Calculating MSE
"""
mse = mean_squared_error(y_test,predictions_e_2)
print(mse)


In [ ]:
mse1 = mean_squared_error(y_test,predictions_e_1)
print(mse1)
mse2 = mean_squared_error(y_test,predictions_e_2)
print(mse2)
mse3 = mean_squared_error(y_test,predictions_e_3)
print(mse3)
mse4 = mean_squared_error(y_test,predictions_e_4)
print(mse4)
mse5 = mean_squared_error(y_test,predictions_e_5)
print(mse5)

In [ ]:
mse6 = mean_squared_error(y_test,predictions_e_6)
print(mse6)
mse7 = mean_squared_error(y_test,predictions_e_7)
print(mse7)
mse8 = mean_squared_error(y_test,predictions_e_8)
print(mse8)
mse9 = mean_squared_error(y_test,predictions_e_9)
print(mse9)
mse10 = mean_squared_error(y_test,predictions_e_10)
print(mse10)

Visuals
---

In [ ]:
# importing the required module
import matplotlib.pyplot as plt
  
# y axis values
y = [mse1,mse2,mse3,mse4,mse5]
# y = y_test[:,1]
# y1 = predictions_e_1[:,1]

# corresponding x axis values
x = [1,10,100,1000,10000]
# x = range(0, 188)

# plotting the points 
# s=3
plt.plot(x, y,color="red", marker = 'o')
# plt.scatter(x, y1,color="blue",s=s)  

# naming the x axis
plt.xlabel('Hyperparameter C value')
# naming the y axis
plt.ylabel('Mean squared error')

plt.text(4000, 7.5, "gamma = 0.0001, epsilon = 0.03") 
plt.title("The effect of Changing the value of C on mean squared error")
plt.grid()

# function to show the plot
plt.show()


In [ ]:
fig.savefig('/content/drive/MyDrive/workfiles_Kaggle/5da138364db8ce0c98bc00f1/F1/ExcelFiles/WIFI_RSSMap/Figures/Changinggamma')

In [ ]:
# Define plot space
fig, ax = plt.subplots(figsize=(5, 6))

# Define x and y axes
ax.plot(x, 
        y,
        marker = 'o')

# Set plot title and axes labels
ax.set(title = "The effect of Changing the value of C on mean squared error",
       xlabel = "Hyperparameter C value",
       ylabel = "Mean squared error")
plt.text(7000, 8, "Gamma = 0.0001, Epsilon = 0.05")
plt.show()